In [ ]:
from __future__ import print_function
import os.path
import dalmatian as dm
import pandas as pd
import sys
from src.CCLE_postp_function import *
sys.path.insert(0, '../JKBio/')
import Datanalytics as da 
import TerraFunction as terra
%load_ext autoreload
%autoreload 2
%load_ext rpy2.ipython
from taigapy import TaigaClient
from IPython.display import Image,display
tc = TaigaClient()
import numpy as np
output_notebook()
from gsheets import Sheets
sheets = Sheets.from_files('~/.client_secret.json', '~/.storage.json')
replace = {'T': 'Tumor', 'N': 'Normal', 'm': 'Unknown', 'L': 'Unknown'}

In [ ]:
workspace1="broad-genomics-delivery/Getz_IBM_CellLines_Exomes"
workspace2="broad-firecloud-ccle/CCLE_DepMap_WES"
workspace3="broad-genomics-delivery/CCLE_DepMap_WES"
refworkspace="broad-firecloud-ccle/DepMap_Mutation_Calling_CGA_pipeline"
source1="ibm"
source2="ccle"
source3="ccle"
sample_set_id = "19Q4"
release = sample_set_id

samplesetname = "19Q4"
release = samplesetname
sheeturl = "https://docs.google.com/spreadsheets/d/115TUgA1t_mD32SnWAGpW9OKmJ2W5WYAOs3SuSdedpX4"
prevname="19Q3"

In [ ]:
%%R
release <- '19Q4'
prevname <- '19Q3'

In [ ]:
refwm.get_samples()

# Processing release {{sample_set_id}}

In [ ]:
wm1 = dm.WorkspaceManager(workspace1)
wm2 = dm.WorkspaceManager(workspace2)
wm3 = dm.WorkspaceManager(workspace3)

refwm = dm.WorkspaceManager(refworkspace)

In [ ]:
## check: add wm1, source1 once I get access to it
newsamples = createDatasetWithNewCellLines(refwm, sample_set_id, [wm2,wm3], [source2, source3], match='ACH', participantslicepos=10, accept_unknowntypes=True, gsfolderto="gs://ccle_bams/wes/")

## update all sample set

In [ ]:
refwm.update_sample_set('All_samples',refwm.get_samples().index.tolist())

# Run the pipeline

In [ ]:
submission_id = refwm.create_submission("CGA_WES_CCLE_Characterization_Pipeline_v0.1_Jul2019", samplesetname,'sample_set',expression='this.samples')
terra.waitForSubmission(refwm, submission_id)

In [ ]:
submission_id = refwm.create_submission("common_variant_filter", samplesetname, 'sample_set', expression='this.samples')
terra.waitForSubmission(refwm, submission_id)

In [ ]:
submission_id = refwm.create_submission("filterMAF_on_CGA_pipeline", samplesetname,'sample_set',expression='this.samples')
terra.waitForSubmission(refwm, submission_id)

In [ ]:
# we use this function to merge 19Q3add in 19Q3
#terra.updateAllSampleSet(refworkspace,'19Q3interim', Allsample_setname='19Q3')

### filtered

In [ ]:
submission_id1 = refwm.create_submission("aggregateMAFs_selectFields_copy", samplesetname)

### unfiltered

In [ ]:
submission_id2 = refwm.create_submission("aggregateMAFs_selectFields_unfiltered", samplesetname)

In [ ]:
terra.waitForSubmission(refworkspace, [submission_id1,submission_id2])

# Downloading from terra

In [ ]:
res = refwm.get_sample_sets().loc[prevname]
res

In [ ]:
filtered = res['filtered_CGA_MAF_aggregated']
unfiltered = res['unfiltered_CGA_MAF_aggregated']

! gsutil cp $filtered "temp/mutation_filtered_terra_merged.txt"
! gsutil cp $unfiltered "temp/mutation_unfiltered_terra_merged.txt"

## retrieving RNAseq MAFs

# postprocessing

In [ ]:
%%R
source('../gkugener/RScripts/load_libraries_and_annotations.R')
load('../gkugener/DM_OMICS/Annotations.RData') 
# There are some cell lines the celllinemapr does not know how to map so we need to load this data object for now (from old datasets)
source('CCLE_postp_function.R')
library(tidyverse)
library(data.table)
library(magrittr)
library(taigr)
library(celllinemapr) # To pull out DepMap_IDs from CCLE_names where needed


In [ ]:
%%R
newly_merged_maf <- readMutations('temp/mutation.mergedmaf.txt')

In [ ]:
%%R
new_release <- createSNPs('temp/mutation.mergedmaf.txt')

In [ ]:
%%R
names(new_release)

## Quick adding

Here, rather than rerunning the entire analysis, because we know we are adding only WES samples, we can download the previous release's MAF, add the samples, update any annotations, and perform any global filters at the end.

First we need to do an additional step of filtering on coverage and number 

In [ ]:
%%R
previous.release.maf <- load.from.taiga(data.name='depmap-mutations-maf-35fe',  data.version=3, data.file=paste0('mutations.',prevname)
if (colnames(previous.release.maf)[1] == 'X1' || colnames(previous.release.maf)[1] == '') {
 previous.release.maf[,1] <- NULL 
}

In [ ]:
%%R
names(previous.release.maf)

In [ ]:
%%R
merged <- addToMainMutation(previous.release.maf, new_release)

## Adding more

In [ ]:
%%R
more <- load.from.taiga(data.name='',  data.version=3, data.file='')
if (colnames(previous.release.maf)[1] == 'X1' || colnames(previous.release.maf)[1] == '') {
 previous.release.maf[,1] <- NULL 
}
merged <- addToMainMutation(merged, more)

In [ ]:
%%R
names(merged)

In [ ]:
%%R
## check if some rows have nans
length(which(is.na(merged$Hugo_Symbol)))

In [ ]:
%%R
dim(merged)

In [ ]:
%%R
filtered <- filterAllelicFraction(merged)

In [ ]:
%%R
filtered <- filterMinCoverage(filtered$merged, filtered$removed_from_maf)

In [ ]:
%%R
merged %<>% dplyr::rename(Tumor_Seq_Allele2=Tumor_Seq_Allele1)

In [ ]:
%%R
clean_annotations <- mergeAnnotations(merged,previous.release.maf)

In [ ]:
%%R
new_release <- addAnnotation(filtered$merged, clean_annotations, colnames(previous.release.maf))

In [ ]:
%%R
# Save the ready to upload file to upload to taiga
write.table(
  new_release, 
  paste0('temp/mutations.', release, '.tsv'), sep = '\t', quote = F, row.names = F)

# Validation

## Compare to previous release

I would run some checks here comparing the results to the previous releases MAF. Namely:

- Count the total number of mutations per cell line, split by type (SNP, INS, DEL)
- Count the total number of mutations observed by position (group by chromosome, start position, end position and count the number of mutations)
- Look at specific differences between the two MAFs (join on DepMap_ID, Chromosome, Start position, End position, Variant_Type). I would do this for WES only

In [ ]:
mutations = pd.read_csv('temp/mutations.'+release+'.tsv', sep='\t')

## Remove less used 

In [ ]:
len(mutations.columns)

In [ ]:
mutations[mutations["Hugo_Symbol"]=="ACOT4"][mutations['Start_position']==74058831].values

In [ ]:
ac_data = mutations[[val for val in mutations.columns.values if '_AC' in val]]

In [ ]:
ac_names = ac_data.columns.values
ac_data = ac_data.values

In [ ]:
ac_data.shape[0]

In [ ]:
allnan

## check important mutations

In [ ]:
# MOLM13, MV411 cell lines- The well known mutation status of FLT3

In [ ]:
# TP53

In [ ]:
toofew = 0
allnan = 0
for pos, val in enumerate(ac_data):
    i = 0
    print(100*pos/ac_data.shape[0],end='\r')
    for p, v in enumerate(val):
        if v is np.nan:
            i+=1
    if i==7:
        mutations = mutations.drop[pos]
        allnan+=1

### basic counts

In [ ]:
#Count the total number of mutations per cell line, split by type (SNP, INS, DEL)


In [ ]:
# Count the total number of mutations observed by position

Are mutation consistent?

In [ ]:
#  to check this, if you group all the mutations in the mutations table by Chromosome, Start_position, End_position, Reference_Allele, Tumor_Seq_Allele1 columns, they should all have the same annotation for the other columns (protein change, exac_af, etc...)


QC mutations, for a known dependency, check if it matches mutation of this gene. (if P53 is mutated, cannot have dependency on P53 or MDM2 MDM4/ inverse fir BRAF and KRAF to themselves)

# uploading on taiga

In [ ]:
tc.update_dataset(dataset_permaname="depmap-mutations-maf-35fe",
                 upload_file_path_dict={'temp/mutations.'+release+'.tsv': 'TableTSV'},
                 dataset_description="# Updating to "+release+ """
# Mutations

filtered and unfiltered mutation files from Broad WES and Sanger WES data mapped to hg19
The MAF file for DepMap that includes all of the latest WES samples. This MAF is generated by merging CCLE (WGS, RNAseq, RD, HC) and Sanger (WES) data.

PORTAL TEAM SHOULD NOT USE THIS: There are lines here that should not make it even to internal. Must use subsetted dataset instead. These data will not make it on the portal starting 19Q1. With the DMC portal, there is new cell line release prioritization as to which lines can be included, so a new taiga dataset will be created containing CN for the portal.

version 1:  In 19Q1 the WES_AC column has been replaced by two columns, VA_WES_AC and CGA_WES_AC. We are currently using the Van Allen and CGA based pipeline to generate mutation calls. The CGA pipeline includes more filtering on the MAFs than VA and has a better INDEL caller. However, some of these filters may be removing some variants of interest that are still capture by the VA pipeline, which is why both a retained for now. DEPRECATED:  Missing the VA_WES_AC, CGA_WES_AC columns
version 2: 19Q1 data
version 3: 19Q2 data. We are no longer using the CCLE_WES_AC column. We are only using the CGA pipeline for mutation calls.
version 4: Updating to 19Q3interim DEPRECATED
version 5: Updating to 19Q3interim DEPRECATED
version 6: Updating to 19Q3interim
version 7: Updating to 19Q3 DEPRECATED
version 8: reparing the missing mutation problem DEPRECATED
version 9: reparing the missing column problem


## ** Internal 19Q4****

Adding 17 new cell lines, 3 reprioritized cell lines. log2(COPY RATIO+1). 
Some cells lines have been flagged as:

 - having bad looking copy ration plots = 
 - Genes having a similar CN value accross all []


genes (gene rpkm):
__Rows__:
__Columns__:
Counts (gene counts):
__Rows__:
__Columns__:
Gene level CN data:
__Rows__:
__Columns__:
 DepMap cell line IDs
 gene names in the format HGNC\_symbol (Entrez\_ID)
DepMap\_ID, Chromosome, Start, End, Num\_Probes, Segment\_Mean
 """)
